# Bibliotecas

In [1]:
import math    
import numpy as np
import matplotlib.pyplot as plt
from numpy import linalg as LA

### Funções

In [2]:
def gradient(f, x, y, h = 1e-6):                    # função gradiente
    gradf = np.zeros((2,1))
    gradf[0] = np.imag(f(complex(x,h),y))/h
    gradf[1] = np.imag(f(x,complex(y,h)))/h
    return gradf

def golden_method(f, delta, x0, y0, epsilon):                    # Golden section method
    
    gsm = (math.sqrt(5) + 1)/2                          
    
    # Phase 01
    
    total = 0
    counter = 0
    f_0 = f(x0,y0)
    cond = True
    gradf = -gradient(f, x0, y0)
    alfa_m = np.zeros(3)
    alfa = 0
    
    while cond:
        total = total + 1
        
        alfa = alfa + delta*gsm**counter
        x = x0 + alfa*gradf[0][0]
        y = y0 + alfa*gradf[1][0]
        f_1 = f(x,y)
        
        if f_1 < f_0:
            f_0 = f_1
            counter = counter + 1
            alfa_m[0] = alfa_m[1]
            alfa_m[1] = alfa_m[2]
            alfa_m[2] = alfa
        else:
            counter = 0    
            cond = False
            alfa_m[0] = alfa_m[1]
            alfa_m[1] = alfa_m[2]
            alfa_m[2] = alfa  
            
     # Phase 02
    
    alfa_range = np.array([alfa_m[0], alfa_m[2]])
    cond = True
    
    while cond:
        total = total + 1
        counter = counter + 1
        
        I = alfa_range[1] - alfa_range[0]
        
        alfa_a = alfa_range[0] + (1 - 1/gsm)*I
        
        x_a = x0 + alfa_a*gradf[0][0]
        y_a = y0 + alfa_a*gradf[1][0]
        
        f_a = f(x_a,y_a)
        
        alfa_b = alfa_range[0] + 1/gsm*I
        
        x_b = x0 + alfa_b*gradf[0][0]
        y_b = y0 + alfa_b*gradf[1][0]
        
        f_b = f(x_b,y_b)
    
        
        if f_a > f_b and I > epsilon:
            alfa_range[0] = alfa_a
            
        elif f_a < f_b and I > epsilon:
            alfa_range[1] = alfa_b
            
        elif I < epsilon:
            cond = False
            alfa = (alfa_a + alfa_b)/2
            x = x0 + alfa*gradf[0][0]
            y = y0 + alfa*gradf[1][0]
            f_1 = f(x,y)
    
    return alfa

In [ ]:
def f(x,y):                                         # função objetivo 
    f = 4*(np.sqrt(x**2+(10-y)**2)-10)**2 + (np.sqrt(x**2+(10+y)**2)-10)**2/2 - 5*(x+y)
    return f

fig, ax = plt.subplots()
size = 200
x = np.linspace(-7.5, 15, 100)
y = np.linspace(-7.5, 15, 100)
X, Y = np.meshgrid(x, y)

F = 4*(np.sqrt(X**2+(10-Y)**2)-10)**2 + (np.sqrt(X**2+(10+Y)**2)-10)**2/2 - 5*(X+Y)
x0 = -4
y0 =  4
contours = plt.contour(X, Y, F, 25)                            # plot função objetivo
plt.scatter(x0,y0,size)
plt.clabel(contours, inline=True, fontsize=15)
fig.set_size_inches(20, 12)

cond = True
counter = 0

I_min = 1e-6
epsilon = 1e-1
delta = 1e-6
H_0 = np.identity(2)
gradf0 = gradient(f, x0, y0)

if LA.norm(gradf0) < epsilon:
    cond = False

while cond:
    counter = counter + 1 
    print("Iteraction {0:5d}: f(x) = {1:8.4f}, x1 = {2:8.2f}, x2 = {3:8.2f}.".format(counter, f(x0,y0), x0, y0))
    d_0 = -np.matmul(LA.inv(H_0),gradf0)
    alfa = golden_method(f, delta, x0, y0, I_min)

    # update the design
    x1 = x0 + alfa*d_0[0][0]
    y1 = y0 + alfa*d_0[1][0]
    plt.scatter(x1,y1,size)
    ax.annotate("",
            xy=(x1, y1), xycoords='data',
            xytext=(x0, y0), textcoords='data',
            arrowprops=dict(arrowstyle="-|>",
                            connectionstyle="arc3",lw=3),
            )
    
    if LA.norm(gradient(f, x1, y1)) < epsilon:
        break

    # update the Hessian
    y_0 = gradient(f, x1, y1) - gradient(f, x0, y0)
    s_0 = alfa*d_0
    gradf1 = gradient(f, x1, y1)
    D_0 = np.matmul(y_0,np.transpose(y_0))/np.matmul(np.transpose(y_0), s_0)[0][0]
    E_0 = np.matmul(gradf0,np.transpose(gradf0))/np.matmul(np.transpose(gradf0), d_0)[0][0]
    H_1 = H_0 + D_0 + E_0

    x0 = x1
    y0 = y1
    H_0 = H_1
    gradf0 = gradf1

plt.xticks(fontsize=20)
plt.yticks(fontsize=20)    
fig.savefig('mp_288_3.png',dpi=300)                              # salvar figura como png (dpi = número de pixels)

Iteraction     1: f(x) =  41.5096, x1 =    -4.00, x2 =     4.00.
Iteraction     2: f(x) =  19.5463, x1 =    -4.74, x2 =     1.82.
Iteraction     3: f(x) =  21.6169, x1 =     1.67, x2 =    -2.02.
Iteraction     4: f(x) =  15.5052, x1 =     1.30, x2 =    -1.68.
Iteraction     5: f(x) =  10.5010, x1 =     1.42, x2 =    -1.45.
Iteraction     6: f(x) =   3.9714, x1 =     1.79, x2 =    -1.15.
Iteraction     7: f(x) =  -2.4658, x1 =     2.33, x2 =    -0.85.
Iteraction     8: f(x) =  -8.7728, x1 =     3.00, x2 =    -0.53.
Iteraction     9: f(x) = -14.8258, x1 =     3.76, x2 =    -0.18.
Iteraction    10: f(x) = -20.3182, x1 =     4.50, x2 =     0.19.
Iteraction    11: f(x) = -25.0861, x1 =     5.16, x2 =     0.60.
Iteraction    12: f(x) = -29.1276, x1 =     5.74, x2 =     1.02.
Iteraction    13: f(x) = -32.4778, x1 =     6.25, x2 =     1.45.
Iteraction    14: f(x) = -35.1776, x1 =     6.70, x2 =     1.88.
Iteraction    15: f(x) = -37.2717, x1 =     7.09, x2 =     2.30.
Iteraction    16: f(x) = 

In [ ]:
gradf0 = gradient(f, x0, y0)
gradf0

In [ ]:
gradf1